In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technicals.indicators import MACD
from simulation.guru_tester import GuruTester

In [3]:
BUY = 1
SELL = -1
NONE = 0

def apply_signal(row):
    if row.direction == BUY and row['mid-l'] > row.EMA_100:
        return BUY
    if row.direction == SELL and row['mid-h'] < row.EMA_100:
        return SELL
    return NONE

In [4]:
pair = "EUR_USD"
df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [5]:
df_an = MACD(df_an)
df_an['EMA_100']=df_an['mid-c'].ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [6]:
def apply_cross(row):
    if row.macd_delta > 0 and row.macd_delta_prev < 0 :
        return BUY
    if row.macd_delta < 0 and row.macd_delta_prev > 0 :
        return SELL
    return NONE

In [7]:
df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
df_an['direction'] = df_an.apply(apply_cross, axis=1)

In [8]:
df_an.direction.value_counts()

direction
 0    13887
-1      598
 1      597
Name: count, dtype: int64

In [9]:
df_an.head()

volume                      time    mid-o    mid-h    mid-l    mid-c  \
0     707 2021-10-07 03:00:00+00:00  1.15546  1.15562  1.15526  1.15556   
1     969 2021-10-07 04:00:00+00:00  1.15555  1.15574  1.15540  1.15556   
2    1328 2021-10-07 05:00:00+00:00  1.15558  1.15569  1.15494  1.15496   
3    4980 2021-10-07 06:00:00+00:00  1.15498  1.15697  1.15490  1.15682   
4    5843 2021-10-07 07:00:00+00:00  1.15684  1.15706  1.15616  1.15688   

     bid-o    bid-h    bid-l    bid-c  ...    ask-h    ask-l    ask-c  \
0  1.15539  1.15555  1.15519  1.15549  ...  1.15570  1.15533  1.15564   
1  1.15548  1.15567  1.15532  1.15549  ...  1.15580  1.15547  1.15564   
2  1.15551  1.15562  1.15487  1.15490  ...  1.15576  1.15501  1.15503   
3  1.15491  1.15690  1.15483  1.15676  ...  1.15704  1.15496  1.15689   
4  1.15677  1.15700  1.15609  1.15682  ...  1.15713  1.15623  1.15695   

       MACD    SIGNAL      HIST   EMA_100  macd_delta  macd_delta_prev  \
0 -0.000560 -0.000816  0.000256  1.158049    0.000256              NaN   
1 -0.000515 -0.000756  0.000241  1.157992    0.000241         0.000256   
2 -0.000521 -0.000709  0.000188  1.157923    0.000188         0.000241   
3 -0.000372 -0.000642  0.000270  1.157898    0.000270         0.000188   
4 -0.000246 -0.000562  0.000317  1.157875    0.000317         0.000270   

   direction  
0          0  
1          0  
2          0  
3          0  
4          0  

[5 rows x 21 columns]

In [10]:
def run_pair(pair, time_d=1):
    df_an = pd.read_pickle(f"../data/{pair}_H{time_d}.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an=MACD(df_an)
    df_an['EMA_100']=df_an['mid-c'].ewm(span=100, min_periods=100).mean()

    df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
    df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
    df_an['direction'] = df_an.apply(apply_cross, axis=1)
    
    our_cols=['time', 'mid-o', 'mid-h', 'mid-l', 'mid-c',
              'bid-o', 'bid-h','bid-l', 'bid-c', 
              'ask-o', 'ask-h', 'ask-l', 'ask-c',
              'direction','EMA_100']
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True,
        time_d=time_d
    )

    gt.run_test()
    return gt.df_results

In [11]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD"]:
    res.append(dict(pair=p, res=run_pair(p, time_d=1)))

In [12]:
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -171.0
EUR_USD -31.5
GBP_JPY -27.5
USD_CAD -93.0


In [13]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD"]:
    res.append(dict(pair=p, res=run_pair(p, time_d=4)))
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -31.5
EUR_USD 14.5
GBP_JPY 24.5
USD_CAD 17.0
